In [1]:
import pandas as pd
from pandas.core.frame import DataFrame
import glob
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.fft import fft, fftfreq
from scipy.signal import firwin, lfilter, butter
# import asrpy
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras import backend as K #转换为张量


2022-06-13 20:22:10.609611: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/neaf-2070/.mujoco/mujoco200_linux/bin:/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:/home/neaf-2070/.mujoco/mujoco200_linux/bin:/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:
2022-06-13 20:22:10.609632: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
with open('labels.pickle', 'rb') as f:
    labels = pickle.load(f)

with open('dataset_Bandpass_tragards.pickle', 'rb') as f:
    dataset = pickle.load(f)


把000000的labelindex挑出來

In [5]:
c_0 = []
c = []
for i in range(0,len(labels)):
    if labels[i].sum() == 0:
        c_0.append(i)
    else:
        c.append(i)

把000000的label跟data分開和其他label跟data分開

In [7]:
label_0 = []
label_1 = []
data_0 = []
data_1 = []
for i in c_0:
    label_0.append(np.array(labels[i]))
    data_0.append(dataset[i])
for i in c:
    label_1.append(np.array(labels[i]))
    data_1.append(dataset[i])

減少屬於000000label 的data數量，使數量跟其他label接近

In [9]:
datax, data_00 = train_test_split(data_0,test_size=0.0035,train_size=1-0.0035)
datax = []
len(data_00)

349

In [10]:
data = data_00+data_1
label = label_0[:len(data_00)]+label_1

In [12]:
data = np.array(data)
label = np.array(label)

打亂訓練數據

In [13]:
datax, data, labelx, label = train_test_split(data, label,test_size=0.99,train_size=0.01)

In [15]:
X_data = data.astype('float64')
Y_data = label.astype('int64')
# X_train = np.array(dataset_fft)
X_data = X_data.reshape(len(X_data), X_data[0].shape[0], X_data[0].shape[1], 1)
# Y_train = np.array(labels)

打亂訓練數據

In [16]:

def new_model():
    model = Sequential()
    model.add(Conv2D(filters = 32, padding = "same", kernel_size = (X_data.shape[1],3), activation = "relu", input_shape = (X_data.shape[1], X_data.shape[2], 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 64, padding = "same", kernel_size = (1,X_data.shape[2]), activation = "relu", input_shape = (X_data.shape[1], X_data.shape[2], 1)))
    model.add(BatchNormalization())
    # model.add(Conv2D(filters = 64, padding = "same", kernel_size = (3,3), activation = "relu", input_shape = (X_data.shape[1], X_data.shape[2], 1)))
    # model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dense(64, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dense(6, activation = "sigmoid"))

    adam = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer = adam, loss = "binary_crossentropy", metrics = ["accuracy"])
    
    return model

In [17]:
data_fft_model = new_model()
data_fft_model.summary()

2022-06-13 20:22:24.540851: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-13 20:22:24.541163: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/neaf-2070/.mujoco/mujoco200_linux/bin:/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:/home/neaf-2070/.mujoco/mujoco200_linux/bin:/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:
2022-06-13 20:22:24.541213: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/neaf-2070/.mujoco/mujoco200_linux/bin:/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:/home/ne

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 14, 32)       9632      
                                                                 
 batch_normalization (BatchN  (None, 100, 14, 32)      128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 14, 64)       28736     
                                                                 
 batch_normalization_1 (Batc  (None, 100, 14, 64)      256       
 hNormalization)                                                 
                                                                 
 flatten (Flatten)           (None, 89600)             0         
                                                                 
 dense (Dense)               (None, 256)               2

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [18]:
train_history = data_fft_model.fit(X_data, Y_data, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10


2022-06-13 20:22:25.142604: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 91750400 exceeds 10% of free system memory.
2022-06-13 20:22:25.151973: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 91750400 exceeds 10% of free system memory.


31/31 [==============================] - 10s 323ms/step - loss: 0.7224 - accuracy: 0.2363 - val_loss: 0.6639 - val_accuracy: 0.3008
Epoch 2/10
31/31 [==============================] - 6s 188ms/step - loss: 0.6028 - accuracy: 0.3880 - val_loss: 0.6032 - val_accuracy: 0.4634
Epoch 3/10
31/31 [==============================] - 6s 188ms/step - loss: 0.5351 - accuracy: 0.4766 - val_loss: 0.5534 - val_accuracy: 0.5061
Epoch 4/10
31/31 [==============================] - 6s 189ms/step - loss: 0.4569 - accuracy: 0.5565 - val_loss: 0.4896 - val_accuracy: 0.5142
Epoch 5/10
31/31 [==============================] - 6s 188ms/step - loss: 0.3753 - accuracy: 0.6319 - val_loss: 0.4431 - val_accuracy: 0.4878
Epoch 6/10
31/31 [==============================] - 6s 188ms/step - loss: 0.3102 - accuracy: 0.6899 - val_loss: 0.4232 - val_accuracy: 0.2947
Epoch 7/10
31/31 [==============================] - 6s 188ms/step - loss: 0.2297 - accuracy: 0.7653 - val_loss: 0.4144 - val_accuracy: 0.3598
Epoch 8/10
31/31

打亂訓練數據

In [19]:
with open('test_labels.pickle', 'rb') as f:
    tlabels = pickle.load(f)
    
with open('test_Bandpass_tragards.pickle', 'rb') as f:
    tdataset = pickle.load(f)

In [21]:
c_0 = []
c = []
for i in range(0,len(tlabels)):
    if tlabels[i].sum() == 0:
        c_0.append(i)
    else:
        c.append(i)

In [22]:
label_0 = []
label_1 = []
data_0 = []
data_1 = []
for i in c_0:
    label_0.append(np.array(tlabels[i]))
    data_0.append(np.array(tdataset[i]))
for i in c:
    label_1.append(np.array(tlabels[i]))
    data_1.append(np.array(tdataset[i]))
    
label_1 = np.array(label_1)
data_1 = np.array(data_1)
label_1 = label_1.astype('float64')
data_1 = data_1.astype('int64')

In [23]:
t_labels = []
t_dataset = []
for i in range(len(tlabels)):
    t_labels.append(np.array(tlabels[i]))
    t_dataset.append(np.array(tdataset[i]))
    
tdataset = np.array(t_dataset)
tlabels = np.array(t_labels)
t_labels = []
t_dataset = []

In [24]:
tdataset = tdataset.astype('float64')
tlabels = tlabels.astype('int64')

In [25]:
scores=data_fft_model.evaluate(tdataset,tlabels)
print()
print('accuracy',scores[1])

556/556 [==============================] - 13s 23ms/step - loss: 0.2156 - accuracy: 0.8343

accuracy 0.8343085646629333


打亂訓練數據

In [26]:
scores=data_fft_model.evaluate(data_1,label_1)
print()
print('accuracy',scores[1])

24/24 [==============================] - 1s 23ms/step - loss: 0.4762 - accuracy: 0.3786

accuracy 0.3786279559135437


In [27]:
# 模型結構存檔
from keras.models import model_from_json
json_string = data_fft_model.to_json()
with open("Bandpass_tra.config", "w") as text_file:
    text_file.write(json_string)

    
# 模型訓練結果存檔
data_fft_model.save_weights("Bandpass_tra.weight")